In [ ]:
# If running from Colab, please uncomment and run the following cell to get summer installed.
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
from jax import numpy as jnp
from datetime import datetime
from summer2 import CompartmentalModel, StrainStratification
from summer2.parameters import Parameter, Function, Time
from plotly import graph_objects as go
pd.options.plotting.backend = 'plotly'

## Multiple competing strains
This notebook applies a two-strain stratification to the base model.
The two strains compete for the same susceptible population.
The assumption that recovered persons are immune to futher infection
has not been adjusted.

### Seeding
So that the two strains are seeded in the same way,
we first ensure that all of the starting population is susceptible.
We then use a triangular function to inject infectious
persons into the model in order to seed the epidemic with each strain.
The time that each strain is introduced is then parameterised for each.
The duration and rate of the seed is relatively unimportant
because most of the dynamics are driven by transmission 
as soon as the prevalence of the new strain reaches significant levels.

### Competition
Because the assumption of complete immunity after infection has not beeen relaxed,
this implies that after recovery,
people are fully immune to infection with either strain.
Given that we introduce the second (`mutant`) strain some time
after the introduction of the first (`wild`) strain
when the epidemic has begun to decline,
there are relatively fewer susceptibles for the second strain to infect.
We therefore increase its infectiousness to very high levels to get
an observable epidemic.
This is unlikely to be the best way to model most competing strain dynamics, 
but illustrates the code structure for implementing multi-strain models.
In practice, and for simulating COVID-19 dynamics,
we would likely wish to allow for subsequent strains to escape 
past immunity.
To achieve this, we would have to implement reinfection processes in the model.

In [ ]:
analysis_start_date = datetime(2021, 1, 1) 
analysis_end_date = datetime(2023, 1, 1)

base_comps = ['susceptible', 'exposed', 'infectious', 'recovered']

epi_model = CompartmentalModel(
    [analysis_start_date, analysis_end_date],
    base_comps,
    ['infectious'],
    ref_date=datetime(2019, 12, 31),
)
epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', 'exposed')
epi_model.add_transition_flow('progression', 1.0 / Parameter('incubation_period'), 'exposed', 'infectious')
epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
epi_model.set_initial_population({'susceptible': 33e6})
incidence = epi_model.request_output_for_flow('incidence', 'progression', save_results=False)
epi_model.request_function_output('notifications', incidence * Parameter('detection_prop'));

In [ ]:
def triangle_wave_func(
    time: float, 
    start: float, 
    duration: float, 
    peak: float,
) -> float:
    """Generate a peaked triangular wave function
    that starts from and returns to zero.

    Args:
        time: Model time
        start: Time at which wave starts
        duration: Duration of wave
        peak: Peak flow rate for wave

    Returns:
        The wave function
    """
    gradient = peak / (duration * 0.5)
    peak_time = start + duration * 0.5
    time_from_peak = jnp.abs(peak_time - time)
    return jnp.where(time_from_peak < duration * 0.5, peak - time_from_peak * gradient, 0.0)

In [ ]:
# Strain stratification
comps_to_stratify = [c for c in base_comps if c != 'susceptible']
strain_strata = ['wild', 'mutant']
strain_strat = StrainStratification('strain', strain_strata, comps_to_stratify)
epi_model.stratify_with(strain_strat)
strain_strat.add_infectiousness_adjustments('infectious', {'wild': None, 'mutant': Parameter('mutant_infectiousness')})
for strain in strain_strata:
    seed_args = [Time, Parameter(f'{strain}_seed_time'), Parameter('seed_duration'), Parameter('seed_peak')]
    voc_seed_func = Function(triangle_wave_func, seed_args)
    epi_model.add_importation_flow(f'seed_{strain}', voc_seed_func, 'infectious', split_imports=True, dest_strata={'strain': strain})
    strain_inc = epi_model.request_output_for_flow(f'incX{strain}', 'progression', dest_strata={'strain': strain}, save_results=False)
    epi_model.request_function_output(f'notifX{strain}', strain_inc * Parameter('detection_prop'))

epoch = epi_model.get_epoch()

In [ ]:
parameters = {
    'contact_rate': 0.4,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'detection_prop': 0.15,
    'wild_seed_time': epoch.datetime_to_number(datetime(2021, 1, 1)),
    'mutant_seed_time': epoch.datetime_to_number(datetime(2021, 7, 1)),
    'seed_duration': 10.0,
    'seed_peak': 10.0,
    'mutant_infectiousness': 4.0,
}
epi_model.run(parameters)
epi_model.get_derived_outputs_df()[[f'notifX{strat}' for strat in strain_strata]].plot.area()